# Word Embedding Vectors

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import os

In [ ]:
BASE_PATH = '/root/work/datasets/'
BASE_SEQUENCE_PATH = os.path.join(BASE_PATH, 'artwork_sequence')
EMBEDDINGS_PATH = os.path.join(BASE_PATH, 'Word Embeddings Pretrained Vectors')

## Load data

In [ ]:
df_metadata = pd.read_csv(os.path.join(BASE_SEQUENCE_PATH, 'all_metadata_with_style_genre.csv'))
df_metadata = df_metadata.drop(columns=['name reverse'])
df_metadata.head()

In [ ]:
df_metadata.shape

In [ ]:
df_all_metadata = pd.read_csv(os.path.join(BASE_PATH, 'train_mayors_style_encoded.csv'))
df_all_metadata = df_all_metadata[['title', 'style', 'genre','date', 'artist']]
df_all_metadata.head()

In [ ]:
df_all_metadata.shape

In [ ]:
all_data_matrix = np.load(os.path.join(BASE_PATH, 'train_mayors_style_encode.npy'))
all_data_matrix.shape

In [ ]:
df_metadata = df_all_metadata

## Load Pre-trained embeddings

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec, KeyedVectors

**Load trained embeddings**

In [ ]:
word2vect_model = KeyedVectors.load_word2vec_format(os.path.join(EMBEDDINGS_PATH, 'GLoVe/glove-w2v.6B.100d.txt'), binary=False)

In [ ]:
vocab = list(word2vect_model.vocab.keys())

**Functions to prepare the data and compute the embedding vector**

In [ ]:
def get_w2vec(soap, word2vect_model):
    # vector space of embeddings
    w2v = np.zeros((word2vect_model.vector_size,))
    for s in soap:
        w2v += word2vect_model.get_vector(s)
    return w2v / len(soap)

In [ ]:
def process_feature(feature, word2vect_model):
    
    if not isinstance(feature, str):
        feature = str(feature)
    feature = feature.split(' ')
    feature_clean = list(filter(lambda x: x in word2vect_model.vocab, feature))
    return feature_clean

**Compute the embedding vector**

In [ ]:
df_metadata['w2v data'] = [[]]*df_metadata.shape[0]
for f in ['style', 'genre']:
    df_metadata['w2v data'] = df_metadata['w2v data'] + df_metadata[f].apply(process_feature, args=(word2vect_model,))

target_embeddings = df_metadata['w2v data'].apply(get_w2vec, args=(word2vect_model,))

target_embeddings = np.stack(target_embeddings)
target_embeddings.shape

In [ ]:
np.save(os.path.join(BASE_PATH, 'train_mayors_style_embedding' ), target_embeddings)